# Build FamaBliss CRSP

In [1]:
DATE = '2024-12-31'

In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import process_wrds_treasury_data

In [4]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


# Fama Bliss

In [5]:
db.describe_table(library='crsp_m_treasuries', table='tfz_mth_fb')

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_mth_fb'
fb_raw = db.raw_sql(SQL_QUERY)
fb = fb_raw.rename(columns={'mcaldt':'date','tmnomprc':'price','tmytm':'yld'})
fb = fb.pivot_table(values='price',index='date',columns='kytreasnox')
fb /= 100
fb.rename(columns={2000047:1, 2000048:2, 2000049:3, 2000050:4, 2000051:5},inplace=True)
fb.columns = fb.columns.astype(int)
fb.columns.name = 'maturity'
fb

Approximately 4355 rows in crsp_m_treasuries.tfz_mth_fb.


maturity,1,2,3,4,5
date,,,,,
1952-06-30,0.981778,0.962531,0.942467,0.926341,0.904183
1952-07-31,0.980948,0.960315,0.939312,0.923089,0.898516
1952-08-29,0.980578,0.959750,0.937819,0.921458,0.895717
1952-09-30,0.980312,0.959098,0.937551,0.920961,0.891676
1952-10-31,0.981108,0.959745,0.937583,0.919290,0.897248
...,...,...,...,...,...
2024-08-30,0.957618,0.925447,0.894010,0.862266,0.831964
2024-09-30,0.961482,0.930623,0.899775,0.868140,0.837951
2024-10-31,0.958667,0.921233,0.884890,0.848150,0.814006


# Save to Excel

In [6]:
outfile = f'../data/famabliss_strips_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:
    fb.to_excel(writer, sheet_name= 'prices', index=True)

***